In [1]:
import xarray as xr
import iris
import numpy as np

iris.FUTURE.netcdf_promote = True

In [2]:
import os
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from scipy.misc import toimage

def list_mogreps_uk(folder, years, months, days, hours, fcsts):
    fs = ['prods_op_mogreps-uk_{:04d}{:02d}{:02d}_{:02d}_00_{:03d}.nc'.format(year, month, day, hour, fcst)
            for year in years for month in months for day in days for hour in hours for fcst in fcsts]
    fs = [os.path.join(folder,f) for f in fs]
    return fs

class PrecipDataset(Dataset):
    def __init__(self, filenames, transform=None):
        filenames.sort()
        self.filenames = filenames
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        try:
            precip = iris.load(self.filenames[idx], 'stratiform_rainfall_amount')[0][0]

            scale_factor = 16
            low_res = np.add.reduceat(np.add.reduceat(precip.data, 
                                                      list(range(0, precip.data.shape[0], scale_factor))),
                                      list(range(0, precip.data.shape[1], scale_factor)), 
                                      axis=1) / scale_factor ** 2

            img = toimage(low_res.data)

            if self.transform:
                img = self.transform(img)

            return img, img
        except:
            return self.__getitem__(np.random.randint(0, len(self.filenames)))

In [3]:
precip = PrecipDataset(list_mogreps_uk('/home/ubuntu/sss/mogreps-uk', [2016], [7,8], [1,2,3], [3], [3]))

In [4]:
precip[5]

(<PIL.Image.Image image mode=L size=27x35 at 0x7F90DD69A7B8>,
 <PIL.Image.Image image mode=L size=27x35 at 0x7F90DD69A7B8>)

In [ ]:
from torchvision import datasets, transforms
precip_train = PrecipDataset(list_mogreps_uk('/home/ubuntu/sss/mogreps-uk', 
                                             [2016], 
                                             [6,7,8], 
                                             list(range(1,22)),
                                             [3,9,15,21],
                                             [3,6]),
                             transform=transforms.ToTensor())

precip_test = PrecipDataset(list_mogreps_uk('/home/ubuntu/sss/mogreps-uk', 
                                             [2016], 
                                             [6,7,8], 
                                             [25],
                                             [9, 21], 
                                             [3]),
                            transform=transforms.ToTensor())

precip_train = list(precip_train)
precip_test = list(precip_test)

In [9]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

args_dict = {'batch_size': 5,
             'epochs': 30,
             'cuda': False,
             'seed': 1,
             'log_interval': 10}
args = type('test', (object,), {})()
args.__dict__.update(args_dict)


torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(precip_train,
                                           batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(precip_test,
                                          batch_size=args.batch_size, shuffle=True, **kwargs)

N = 945
n2 = int(N/2)
n3 = int(N/9)
# n2 = 3200
# n3 = 100

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(N, n2)
        self.fc21 = nn.Linear(n2, n3)
        self.fc22 = nn.Linear(n2, n3)
        self.fc3 = nn.Linear(n3, n2)
        self.fc4 = nn.Linear(n2, N)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, N))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if args.cuda:
    model.cuda()


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, N))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= args.batch_size * N

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        print(batch_idx)
        data = Variable(data)
        if args.cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        print(i)
        if args.cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        if i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                  recon_batch.view(args.batch_size, 1, 35, 27)[:n]])
            save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64, n3))
    if args.cuda:
        sample = sample.cuda()
    sample = model.decode(sample).cpu()
    save_image(sample.data.view(64, 1, 35, 27),
               'results/sample_' + str(epoch) + '.png')

0
Train Epoch: 1 [0/504 (0%)]	Loss: 0.139500
1
2
3
4
5
6
7
8
9
10
Train Epoch: 1 [50/504 (10%)]	Loss: 0.065679
11
12
13
14
15
16
17
18
19
20
Train Epoch: 1 [100/504 (20%)]	Loss: 0.033334
21
22
23
24
25
26
27
28
29
30
Train Epoch: 1 [150/504 (30%)]	Loss: 0.025896
31
32
33
34
35
36
37
38
39
40
Train Epoch: 1 [200/504 (40%)]	Loss: 0.039202
41
42
43
44
45
46
47
48
49
50
Train Epoch: 1 [250/504 (50%)]	Loss: 0.027000
51
52
53
54
55
56
57
58
59
60
Train Epoch: 1 [300/504 (59%)]	Loss: 0.038333
61
62
63
64
65
66
67
68
69
70
Train Epoch: 1 [350/504 (69%)]	Loss: 0.023200
71
72
73
74
75
76
77
78
79
80
Train Epoch: 1 [400/504 (79%)]	Loss: 0.029420
81
82
83
84
85
86
87
88
89
90
Train Epoch: 1 [450/504 (89%)]	Loss: 0.033046
91
92
93
94
95
96
97
98
99
100
Train Epoch: 1 [400/504 (99%)]	Loss: 0.022006
====> Epoch: 1 Average loss: 0.0408
0
1
====> Test set loss: 0.1062
0
Train Epoch: 2 [0/504 (0%)]	Loss: 0.026968
1
2
3
4
5
6
7
8
9
10
Train Epoch: 2 [50/504 (10%)]	Loss: 0.023182
11
12
13
14
15
16
17
18
1